# Aula #26 – Processamento de Linguagem Natural & Análise de Sentimento

# Word2vec

Já vimos antes que é possível transformar um texto em _features_ numéricas. Uma sofisticação do método _Bag of words_ é incorporar o contexto das palavras vizinhas nessas _features_ (é comum chamar o vetor de _features_ numéricas de _embedding_).

Imagine que nossa janela de contexto (context window) tem tamanho 5 (2 palavras _antes_ e 2 palavras _depois_ da palavra _central_).

Então, se a frase fosse `The quick brown fox jumps over the lazy dog`, teríamos as seguintes janelas:

<img src="data/nb_figs/windows_word2vec.png" width="600"/>

Para cada uma das janelas formadas, temos o vetor correspondente a elas (usando o _Bag of words_ binário - com apenas 0s e 1s; também chamado de `one-hot encoding`):

<img src="data/nb_figs/one_hot_encoding_word2vec.png" width="600"/>

Há duas arquiteturas possíveis para se obter os `embeddings` word2vec. Uma delas é chamada de `CBoW` (_Continuous Bag of Words_) e outra é chamada de `Skip gram`. Aqui, vamos focar no `Skip gram`, que considera como input o vetor da palavra central da janela, e como output, os vetores do contexto. O objetivo do algoritmo é aprender os pesos da _hidden layer_, de forma que as probabilidades finais sejam condizentes com as co-ocorrências das palavras em nosso _corpus_ de documentos.

<img src="data/nb_figs/nn_word2vec_large.png" width="800"/>

Ao final do treinamento, a matriz correspondente à _hidden layer_, com 10 mil (tamanho do vocabulário) linhas e 300 (quantidade de dimensões do _embedding_) colunas será tal que cada linha representará o embedding de uma palavra do vocabulário.

Para saber mais sobre `word2vec`, leia em:

* http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
* https://nathanrooy.github.io/posts/2018-03-22/word2vec-from-scratch-with-python-and-numpy/
* https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/

## Similaridade entre ingredientes - uma aplicação do _word2vec_ a um dataset de receitas

O dataset utilizado aqui compreende os datasets de treino e teste do [Recipe Ingredients Dataset do Kaggle](https://www.kaggle.com/kaggle/recipe-ingredients-dataset).

A ideia é treinar um modelo `word2vec` usando a biblioteca [gensim](https://radimrehurek.com/gensim/index.html) e depois construirmos uma aplicação pela qual seja possível obter uma lista dos ingredientes mais similares a um determinado ingrediente. Vamos tentar?

### Leitura do dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('data/datasets/kaggle_recipes/recipes.json').reset_index(drop=True)

In [3]:
df.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,italian,5875,"[pimentos, sweet pepper, dried oregano, olive ..."
3,italian,17636,"[tomato sauce, shredded carrots, spinach, part..."
4,italian,36837,"[marinara sauce, goat cheese, minced garlic, s..."


## Treinamento do _word2vec_

**Tarefa:** Treine um modelo word2vec usando os dados da coluna `ingredients` (`words_list`)

Dica: Leia a documentação sobre a classe `Word2Vec`

In [ ]:
?Word2Vec

In [ ]:
from gensim.models import Word2Vec

In [ ]:
words_list = df['ingredients'].tolist()

In [ ]:
%%time
model = ### complete

### Similaridade entre vetores

Em modelos vetoriais de linguagem, em geral, utiliza-se a similaridade de cosseno como medida de similaridade entre dois vetores, já que ela captura a noção de que vetores apontando para a mesma direção são próximos.

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
gelato_vec = model.wv['gelato']
sorbet_vec = model.wv['sorbet']

In [ ]:
def similarity_between_vec(vec1, vec2):
    return 1 - cosine(vec1, vec2)

In [ ]:
similarity_between_vec(gelato_vec, sorbet_vec)

### Termos mais comuns

Vamos ver quais são os termos mais comuns do dataset?

In [ ]:
from collections import Counter

In [ ]:
all_ingredients = sum(df['ingredients'].tolist(), [])

In [ ]:
Counter(all_ingredients).most_common(20)

### Os mais próximos

Um método legal do objeto `Word2VecKeyedVectors` é o `most_similar`, que retorna as palavras mais similares a uma determinada palavra. Note que podemos modificar a quantidade de itens retornados, colocando um valor para parâmetro `topn` (por padrão, ele é 10).

**Tarefa:** brinque até ficar satisfeito.

As relações fazem sentido?

In [ ]:
?model.wv.most_similar

In [ ]:
model.wv.most_similar('rice')

**Tarefa:** Imprima os mais similares para os 5 ingredientes mais comuns obtidos anteriormente. 

In [ ]:
### complete

## Visualização das relações entre os ingredientes

Vamos agora construir uma ferramenta que permite:

1. buscar o nome de um ingrediente
2. retornar os ingredientes mais próximos (que não são ele mesmo)

Para fazer isso, vamos usar novamente o recurso `widgets`. A função [interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html#Basic-interact) permite que ao digitar o nome do ingrediente, já iniciemos a busca por ele na lista de ingredientes disponível (variável `VOCAB` declarada abaixo). Ela também vai nos permitir mostrar os ingredientes disponíveis retornados pela busca e garantir que quando for selecionado um ingrediente, seja disparado o cálculo dos ingredientes mais próximos.

In [ ]:
from ipywidgets import interact, widgets

In [ ]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

In [ ]:
VOCAB = set(model.wv.vocab.keys())

**Tarefa:** Complete a função abaixo, que dado um ingrediente (`ingredient`), retorna os ingredientes mais similares, sem mostrar os ingredientes que contêm o nome do ingrediente de input (ou seja, `pasta` não deve retornar `farfalle pasta`, por exemplo).

In [ ]:
def get_similar(ingredient):
    """Returns the most similar ingredients to a selected `ingredient`,
        excluding ingredients which contain the name of the ingredient
    """
    ### complete

In [ ]:
def show_most_similar(ingredient):
    """Print the most similar ingredients to a selected `ingredient`
    """
    if ingredient == '':
        printmd('')
    else:
        for w in get_similar(ingredient):
            printmd(f'* {w}')

In [ ]:
def search_text(text):
    style = {'description_width': 'initial'}
    options = [v for v in VOCAB if text.lower() in v]
    if text in options:
        options.remove(text)
        options = [text] + sorted(options)
    dropdown_widget = widgets.Dropdown(
        options=options,
        description='Available ingredient:',
        disabled=False,
        style=style
    )
    interact(show_most_similar, ingredient=dropdown_widget)

In [ ]:
w = widgets.Text(
    value='peanut butter',
    description='Type ingredient name:',
    disabled=False,
    style={'description_width': 'initial'}
)

**Tarefa:** Teste nossa recém-construída ferramenta e verifique se existem normalizações no texto que você acharia bom fazer.

In [ ]:
interact(search_text, text=w);

**Tarefa Bônus:** Implemente um filtro por tipo de culinária (chinesa, italiana, grega etc.), para que os ingredientes exibidos como similares façam parte do tipo de culinária escolhida.